### 遺產稅試算：

### 114年的新版本

In [14]:
import random
import time

# Selenium 相關模組
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException

# Z3 模組
from z3 import Solver, Real, If, And, sat, ToInt

# ---------------------------
# 計算器 function (使用 Z3 求解)
# ---------------------------
def estate_tax_z3_solver(
    # 使用者輸入：死亡日期區間、是否為軍警公教因公死亡
    death_period: int,
    is_military_police: bool,
    
    # 遺產總額相關
    land_value: float,           
    building_value: float,       
    house_value: float,          
    deposit_bonds_value: float,  
    stock_invest_value: float,   
    cash_gold_jewelry_value: float,
    gift_in_2yrs_value: float,   
    
    # 扣除額（人數或金額）
    spouse_count: int,            
    lineal_descendant_count: int,  
    father_mother_count: int,      
    disability_count: int,         
    dependent_count: int,          

    farmland_val: float,           
    inheritance_6to9_val: float,   
    unpaid_tax_fines_val: float,   
    unpaid_debts_val: float,       
    will_management_fee: float,    
    public_facility_retention_val: float,
    spouse_surplus_right_val: float,
    
    # 扣抵稅額
    gift_tax_offset: float,    
    foreign_tax_offset: float  
):
    """
    death_period 對應：
      1 -> 民國103年1月1日 ~ 106年5月11日
      2 -> 民國106年5月12日 ~ 110年12月31日
      3 -> 民國111年1月1日  ~ 112年12月31日
      4 -> 民國113年1月1日(含) ~ 113年12月31日(或113年後之規則)
      5 -> 民國114年1月1日(含)以後

    依據：
    - 免稅額：
      (1)(2) => 12,000,000  
      (3)(4)(5) => 13,330,000  
      若 is_military_police=True => 免稅額加倍 (分別變成 24,000,000 與 26,660,000)。
    
    - 喪葬費：
      (1)(2)(3) => 1,230,000  
      (4)(5)   => 1,380,000

    - 稅率與累進差額：
      (A) death_period = 1：單一稅率 10%，累進差額 0。
      (B) death_period = 2,3,4：
          若課稅遺產淨額 ≤ 50,000,000 元，稅率 10%，累進差額 0；
          若介於 50,000,001 ~ 100,000,000 元，稅率 15%，累進差額 2,500,000；
          若超過 100,000,000 元，稅率 20%，累進差額 7,500,000。
      (C) death_period = 5：
          若課稅遺產淨額 ≤ 56,210,000 元，稅率 10%，累進差額 0；
          若介於 56,210,001 ~ 112,420,000 元，稅率 15%，累進差額 2,810,500；
          若超過 112,420,000 元，稅率 20%，累進差額 8,431,500。
    """

    # (A) 根據 death_period 決定免稅額與喪葬費基準金額
    if death_period in [1, 2]:
        base_exemption_val = 12000000
        funeral_val = 1230000
    else:
        base_exemption_val = 13330000
        funeral_val = 1230000
        if death_period in [4, 5]:
            funeral_val = 1380000

    # 若為軍警公教因公死亡 => 免稅額加倍
    if is_military_police:
        base_exemption_val *= 2

    s = Solver()

    # (B) 建立 Z3 變數
    inheritance_total = Real('inheritance_total')
    base_exemption = Real('base_exemption')
    total_deduction = Real('total_deduction')
    taxable_inheritance = Real('taxable_inheritance')
    estate_tax = Real('estate_tax')
    
    # (C) 遺產總額 = 各項資產之和
    s.add(inheritance_total ==
          land_value + building_value + house_value
          + deposit_bonds_value + stock_invest_value
          + cash_gold_jewelry_value + gift_in_2yrs_value)
    
    # (D) 帶入免稅額與喪葬費
    s.add(base_exemption == base_exemption_val)
    
    # (E) 設定各項扣除額（依據《遺產及贈與稅法第17條》）
    spouse_deduction_val        = Real('spouse_deduction_val')
    lineal_deduction_val        = Real('lineal_deduction_val')
    father_mother_deduction_val = Real('father_mother_deduction_val')
    disability_deduction_val    = Real('disability_deduction_val')
    dependent_deduction_val     = Real('dependent_deduction_val')
    funeral_deduction_val       = Real('funeral_deduction_val')
    
    s.add(
        spouse_deduction_val ==
        spouse_count * If(death_period == 4 or death_period == 5, 5530000, 4930000)
    )
    s.add(
        lineal_deduction_val ==
        lineal_descendant_count * If(death_period == 4 or death_period == 5, 560000, 500000)
    )
    s.add(
        father_mother_deduction_val ==
        father_mother_count * If(death_period == 4 or death_period == 5, 1380000, 1230000)
    )
    s.add(
        disability_deduction_val ==
        disability_count * If(death_period == 4 or death_period == 5, 6930000, 6180000)
    )
    s.add(
        dependent_deduction_val ==
        dependent_count * If(death_period == 4 or death_period == 5, 560000, 500000)
    )
    s.add(funeral_deduction_val == funeral_val)
    
    # (F) 扣除額合計
    s.add(total_deduction ==
          spouse_deduction_val
        + lineal_deduction_val
        + father_mother_deduction_val
        + disability_deduction_val
        + dependent_deduction_val
        + farmland_val
        + inheritance_6to9_val
        + unpaid_tax_fines_val
        + unpaid_debts_val
        + funeral_deduction_val
        + will_management_fee
        + public_facility_retention_val
        + spouse_surplus_right_val
    )

    # (G) 課稅遺產淨額
    s.add(taxable_inheritance == inheritance_total - base_exemption - total_deduction)
    
    # (H) 設定稅率與累進差額
    bracket_rate = Real('bracket_rate')
    bracket_diff = Real('bracket_diff')
    
    s.add(
        If(
            death_period == 1,
            And(bracket_rate == 0.10, bracket_diff == 0),
            If(
                death_period == 5,
                If(
                    taxable_inheritance <= 56210000,
                    And(bracket_rate == 0.10, bracket_diff == 0),
                    If(
                        taxable_inheritance <= 112420000,
                        And(bracket_rate == 0.15, bracket_diff == 2810500),
                        And(bracket_rate == 0.20, bracket_diff == 8431500)
                    )
                ),
                If(
                    taxable_inheritance <= 50000000,
                    And(bracket_rate == 0.10, bracket_diff == 0),
                    If(
                        taxable_inheritance <= 100000000,
                        And(bracket_rate == 0.15, bracket_diff == 2500000),
                        And(bracket_rate == 0.20, bracket_diff == 7500000)
                    )
                )
            )
        )
    )
    
    # (I) 扣抵稅額
    offsets = Real('offsets')
    s.add(offsets == gift_tax_offset + foreign_tax_offset)
    
    # (J) 最終應納遺產稅額：若計算結果為負則取 0
    s.add(
        estate_tax == If(
            taxable_inheritance <= 0,
            0,
            If(taxable_inheritance * bracket_rate - bracket_diff - offsets < 0,
               0,
               taxable_inheritance * bracket_rate - bracket_diff - offsets)
        )
    )
    
    # (K) 求解並轉換為整數
    result = s.check()
    if result == sat:
        model = s.model()
        estate_tax_int = model.evaluate(ToInt(estate_tax))
        return int(estate_tax_int.as_long())
    else:
        return None

### Selenium測試

In [ ]:
# ---------------------------
# Selenium 測試程式碼
# ---------------------------
def run_selenium_test():
    # 測試網頁 URL (請依實際環境修改)
    url = "https://www.etax.nat.gov.tw/etwmain/etw158w/83"
    
    # 設定死亡日期下拉選單對應映射 (網頁 option value 與本地 death_period 的對應)
    web_death_mapping = {
        "3": 1,
        "5": 2,
        "6": 3,
        "7": 4,
        "8": 5
    }
    
    mismatch_count = 0
    match_count = 0
    driver = webdriver.Chrome()
    driver.get(url)
    
    # 輔助函數：填寫欄位後暫停
    def slow_fill(element_id, value, delay=0.3):
        elem = driver.find_element(By.ID, element_id)
        elem.clear()
        elem.send_keys(str(value))
        time.sleep(delay)
    
    # 測試重複 100 次
    for i in range(100):
        # 隨機產生測試參數
        web_death_value = random.choice(list(web_death_mapping.keys()))
        death_period = web_death_mapping[web_death_value]
        
        # 隨機產生 is_military_police 參數
        is_military_police = random.choice([True, False])
        
        # 隨機產生各項資產
        land_value = random.randint(0, 50000000)
        building_value = random.randint(0, 50000000)
        house_value = random.randint(0, 50000000)
        deposit_bonds_value = random.randint(0, 50000000)
        stock_invest_value = random.randint(0, 50000000)
        cash_gold_jewelry_value = random.randint(0, 50000000)
        gift_in_2yrs_value = random.randint(0, 50000000)
        
        # 隨機產生扣除額參數
        spouse_count = random.randint(0, 1)
        lineal_descendant_count = random.randint(0, 3)
        father_mother_count = random.randint(0, 2)
        disability_count = random.randint(0, 2)
        dependent_count = random.randint(0, 3)
        
        farmland_val = random.randint(0, 10000000)
        inheritance_6to9_val = random.randint(0, 10000000)
        unpaid_tax_fines_val = random.randint(0, 10000000)
        unpaid_debts_val = random.randint(0, 10000000)
        will_management_fee = random.randint(0, 10000000)
        public_facility_retention_val = random.randint(0, 10000000)
        spouse_surplus_right_val = random.randint(0, 10000000)
        
        gift_tax_offset = random.randint(0, 10000000)
        foreign_tax_offset = random.randint(0, 10000000)
        
        # 使用計算器 function 進行本地計算
        local_result = estate_tax_z3_solver(
            death_period, is_military_police,
            land_value, building_value, house_value,
            deposit_bonds_value, stock_invest_value,
            cash_gold_jewelry_value, gift_in_2yrs_value,
            spouse_count, lineal_descendant_count,
            father_mother_count, disability_count,
            dependent_count,
            farmland_val, inheritance_6to9_val,
            unpaid_tax_fines_val, unpaid_debts_val,
            will_management_fee, public_facility_retention_val,
            spouse_surplus_right_val,
            gift_tax_offset, foreign_tax_offset
        )
        
        try:
            # 填寫死亡日期下拉選單 (id="typeW6")
            select_elem = Select(driver.find_element(By.ID, "typeW6"))
            select_elem.select_by_value(web_death_value)
            time.sleep(0.5)
            
            # 計算免稅額：依據程式邏輯
            if death_period in [1, 2]:
                base_exemption_val = 12000000
            else:
                base_exemption_val = 13330000
            if is_military_police:
                base_exemption_val *= 2
            
            # 將計算出之免稅額填入「allowances」欄位
            slow_fill("allowances", base_exemption_val)
            
            # 填寫資產相關欄位
            slow_fill("land", land_value)
            slow_fill("groundObjects", building_value)
            slow_fill("houses", house_value)
            
            slow_fill("deposit", deposit_bonds_value)
            slow_fill("stock", stock_invest_value)
            slow_fill("cash", cash_gold_jewelry_value)
            slow_fill("gift", gift_in_2yrs_value)
            
            # 填寫扣除額(人數)
            slow_fill("deduction1", spouse_count)
            slow_fill("deduction2", lineal_descendant_count)
            slow_fill("deduction3", father_mother_count)
            slow_fill("deduction4", disability_count)
            slow_fill("deduction5", dependent_count)
            
            # 填寫其他扣除額(金額)
            slow_fill("deduction6", farmland_val)
            slow_fill("deduction7", inheritance_6to9_val)
            slow_fill("deduction8", unpaid_tax_fines_val)
            slow_fill("deduction9", unpaid_debts_val)
            slow_fill("deduction11", will_management_fee)
            slow_fill("deduction12", public_facility_retention_val)
            slow_fill("deduction13", spouse_surplus_right_val)
            
            # 填寫扣抵稅額欄位
            slow_fill("ica1", gift_tax_offset)
            slow_fill("ica2", foreign_tax_offset)
            
            # 點擊「計算」按鈕 (以 title="計算" 的按鈕定位)
            driver.find_element(By.XPATH, "//button[@title='計算']").click()
            
            # 等待網頁完成計算
            time.sleep(3)
            
            # 讀取網頁上最後結果 (假設結果位於 <span class="ml-2 ml-2 text-danger">)
            result_elem = driver.find_element(By.CSS_SELECTOR, "span.text-danger")
            web_result_str = result_elem.text.strip()
            try:
                web_result = int(web_result_str.replace(",", ""))
            except ValueError:
                web_result = None
            
            # 若不一致則印出所有參數資訊
            if local_result == web_result:
                match_count += 1
                print(f"[一致案例 #{i+1}] 本地計算結果: {local_result}  vs  網頁結果: {web_result}")
            else:
                mismatch_count += 1
                print(f"【不一致案例 #{i+1}】")
                print("參數：")
                print(f"  death_period (web: {web_death_value}, local: {death_period}), is_military_police: {is_military_police}")
                print(f"  land_value: {land_value}, building_value: {building_value}, house_value: {house_value}")
                print(f"  deposit_bonds_value: {deposit_bonds_value}, stock_invest_value: {stock_invest_value}")
                print(f"  cash_gold_jewelry_value: {cash_gold_jewelry_value}, gift_in_2yrs_value: {gift_in_2yrs_value}")
                print(f"  spouse_count: {spouse_count}, lineal_descendant_count: {lineal_descendant_count}, father_mother_count: {father_mother_count}")
                print(f"  disability_count: {disability_count}, dependent_count: {dependent_count}")
                print(f"  farmland_val: {farmland_val}, inheritance_6to9_val: {inheritance_6to9_val}")
                print(f"  unpaid_tax_fines_val: {unpaid_tax_fines_val}, unpaid_debts_val: {unpaid_debts_val}")
                print(f"  will_management_fee: {will_management_fee}, public_facility_retention_val: {public_facility_retention_val}")
                print(f"  spouse_surplus_right_val: {spouse_surplus_right_val}")
                print(f"  gift_tax_offset: {gift_tax_offset}, foreign_tax_offset: {foreign_tax_offset}")
                print(f"本地計算結果: {local_result}  vs  網頁結果: {web_result}")
                print("-" * 50)
                
        except NoSuchElementException as e:
            print(f"在案例 #{i+1} 發生例外: {e}")
        
        # 每次測試結束後等待，再重新載入頁面清空欄位
        time.sleep(1)
        driver.get(url)
    
    print(f"共測試 100 次，一致次數: {match_count}，不一致次數: {mismatch_count}")
    driver.quit()

# ---------------------------
# 主程式執行入口
# ---------------------------
if __name__ == "__main__":
    run_selenium_test()


[一致案例 #1] 本地計算結果: 8277888  vs  網頁結果: 8277888
[一致案例 #2] 本地計算結果: 6639378  vs  網頁結果: 6639378
[一致案例 #3] 本地計算結果: 3875609  vs  網頁結果: 3875609
[一致案例 #4] 本地計算結果: 0  vs  網頁結果: 0
[一致案例 #5] 本地計算結果: 5658064  vs  網頁結果: 5658064
[一致案例 #6] 本地計算結果: 15534784  vs  網頁結果: 15534784
[一致案例 #7] 本地計算結果: 1760611  vs  網頁結果: 1760611
[一致案例 #8] 本地計算結果: 12878411  vs  網頁結果: 12878411
[一致案例 #9] 本地計算結果: 2638622  vs  網頁結果: 2638622
[一致案例 #10] 本地計算結果: 13070762  vs  網頁結果: 13070762
[一致案例 #11] 本地計算結果: 12155486  vs  網頁結果: 12155486
[一致案例 #12] 本地計算結果: 14587642  vs  網頁結果: 14587642
[一致案例 #13] 本地計算結果: 11546306  vs  網頁結果: 11546306
[一致案例 #14] 本地計算結果: 7718059  vs  網頁結果: 7718059
[一致案例 #15] 本地計算結果: 4914793  vs  網頁結果: 4914793
[一致案例 #16] 本地計算結果: 21602187  vs  網頁結果: 21602187
[一致案例 #17] 本地計算結果: 17620101  vs  網頁結果: 17620101
[一致案例 #18] 本地計算結果: 4984736  vs  網頁結果: 4984736
[一致案例 #19] 本地計算結果: 339900  vs  網頁結果: 339900
[一致案例 #20] 本地計算結果: 0  vs  網頁結果: 0
[一致案例 #21] 本地計算結果: 9844918  vs  網頁結果: 9844918
[一致案例 #22] 本地計算結果: 0  vs  網頁結果: 0
[一致案例 #23] 本地計算結果: 

### 不一樣的案例重新測：網頁：11,687,096

In [21]:
estate_tax_z3_solver(
    death_period=5,
    is_military_police=False,
    land_value=18108076,
    building_value=40956673,
    house_value=38354630,
    deposit_bonds_value=10027576,
    stock_invest_value=35877803,
    cash_gold_jewelry_value=6824046,
    gift_in_2yrs_value=45714011,
    spouse_count=0,
    lineal_descendant_count=0,
    father_mother_count=0,
    disability_count=1,
    dependent_count=1,
    farmland_val=1101867,
    inheritance_6to9_val=853093,
    unpaid_tax_fines_val=1893020,
    unpaid_debts_val=6859916,
    will_management_fee=5088145,
    public_facility_retention_val=4802310,
    spouse_surplus_right_val=2298856,
    gift_tax_offset=6638306,
    foreign_tax_offset=3396219
    
)

11687096